# Tutorial Advanced Noten: Suche von Intervallfolgen

(version 2022_01_03)

Dieses Tutorial führt in die Suche von Intervallfolgen auf der Grundlage von CAMAT (Computer-Assisted Music Analysis Tool) ein.

Das Durcharbeiten und Nachvollziehen des Tutoriums soll es Ihnen ermöglichen, selbst Intervallfolgen in eigenen Musikbeispielen (Notendateien) zu suchen.

In [1]:
import sys
import os
sys.path.append(os.getcwd().replace(os.path.join('music_xml_parser', 'ipynb'), ''))
import music_xml_parser as mp
from music21 import * 
import csv
from IPython.display import HTML, display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Mit diesen Befehlen wird nicht nur der CAMAT music_xml_parser, 
# sondern darüber hinaus die Bibliotheken 'numpy' und 'pandas' für statische Auswertungen, 
# 'music21' sowieund 'matplotlib' für grafische Darstellungen geladen. 

# Mit den folgenden Befehlen wird die Formtierung für die Tabellen festgelegt, 
# die weiter unten dargestellt werden - '9999' ist der Maximalwert: 

    
environment.set('autoDownload', 'allow')
# Hierdurch wird der Download von xml-Dateien aus dem Internet ermöglicht. 

Zunächst muss eine Datei geladen werden und mit einem Variablennamen (hier: 'xml_files') bezeichnet werden: 

In [2]:
xml_file = 'https://analyse.hfm-weimar.de/database/03/MoWo_K171_COM_1-4_StringQuar_003_00867.xml'

Mit dem folgenden Befehl wird eine Suchanfrage generiert und anschließend eine Ergebnisliste angezeigt. 
Die gesuchte Intervallfolge wird in Halbtonschritten angegeben; die einzelnen Intervallschritte werden durch ein Komma getrennt:

    interval=[2, 2]
Dadurch wird nach einer Folge von zwei aufsteigenden großen Sekunden (Ganztönen) gesucht. 

Will man nach einer kleinen Terz aufwärts gefolgt von kleiner Sekunde (Halbton) abwärts suchen, muss folgendes eingegeben werden: 

    interval=[3, -1]

Die Ergebnisse können durch den Zusatz 'return_details' unterschiedlich detailliert dargestellt werden. Bei 'False' werden nur die Häufigkeiten der gesuchten Intervallfolge bezogen auf verschiedene Anfangstöne angezeigt. Bei 'True' wird das genaue Vorkommen angezeigt: in welcher Stimme, mit welchem Anfangston, in welchem Takt und bei welcher Zählzeit. 

    return_details=False
    return_details=True


In [3]:
results_01 = mp.core.search.simple_interval_search(xml_file,
                                                   interval=[2, 2],
                                                   return_details=False)

results_01  # durch diesen Befehl wird eine einfach Tabelle dargestellt. 

File at:  /Users/chris/DocumentLocal/workspace/hfm/scripts_in_progress/xml_parser/music_xml_parser/data/xmls_to_parse/hfm_database/MoWo_K171_COM_1-4_StringQuar_003_00867.xml


,Pitch,Occurance
0,F,4
1,A-1,4
2,E-1,11
3,B-1,11


Die Tonhöhen werden hier folgendermaßen benannt: 
- B entspricht H
- 1 = mit #-Vorzeichen
- -1 = mit b-Vorzeichen 
- -2 = mit doppeltem b-Vorzeichen usw.
- 0 = mit Auflösungszeichen 

Hier eine andere Suchanfrage - diesmal mit detaillierterer Tabelle ('return_details=True'):

In [4]:
results_02 = mp.core.search.simple_interval_search(xml_file, 
                                                   interval=[3, -1],
                                                   return_details=True)
results_02

File at:  /Users/chris/DocumentLocal/workspace/hfm/scripts_in_progress/xml_parser/music_xml_parser/data/xmls_to_parse/hfm_database/MoWo_K171_COM_1-4_StringQuar_003_00867.xml


,Pitch,Octave,MIDI,PartName,PartID,Measure,LocalOnset,Onset
0,C,4,60.0,Violino II,2,13,0.5,48.5
1,F,4,65.0,Violino II,2,154,0.5,484.5
2,F,3,53.0,Viola,3,141,0.0,434.0


In [5]:
results_02 = mp.core.search.simple_interval_search(xml_file, interval=[2, 2, 1],return_details=True)


File at:  /Users/chris/DocumentLocal/workspace/hfm/scripts_in_progress/xml_parser/music_xml_parser/data/xmls_to_parse/hfm_database/MoWo_K171_COM_1-4_StringQuar_003_00867.xml


Durch den Befehl "columns" können die Tabellenspalten umbenannt werden: 

In [6]:
mp.utils.display_table(data=results_02, 
                       columns=['Anfangston', 'Oktave','Midi', 'Stimme', 'PartID', 'Takt', 'LokalTon','Ton'])

Anfangston,Oktave,Midi,Stimme,PartID,Takt,LokalTon,Ton
E-1,4,63.0,Violino II,2,12,0.75,44.75
B-1,3,58.0,Violino II,2,44,1.0,144.0
B-1,3,58.0,Violino II,2,46,1.0,150.0
E-1,4,63.0,Violino II,2,108,1.0,336.0
E-1,4,63.0,Violino II,2,110,1.0,342.0
B-1,3,58.0,Violino II,2,139,2.0,430.0
E-1,4,63.0,Violino II,2,155,0.75,488.75
F,3,53.0,Violoncello,4,36,0.0,119.0
F,3,53.0,Violoncello,4,38,0.0,125.0
F,2,41.0,Violoncello,4,44,0.0,143.0
